In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
from interlab.queries import query_for_json, query_model
import langchain_community.chat_models
from treetrace import TracingNode, FileStorage
from interlab_zoo.persona.personas import Factory
from interlab_zoo.persona.contacts import link
from interlab_zoo.persona.channels import ChatChannel
from interlab_zoo.resolve.resolve import Situation, and_join
from serde.yaml import from_yaml
import pandas as pd
import langchain
import xml.etree.ElementTree as ET
import uuid
import itertools

dotenv.load_dotenv()

True

In [3]:
#from langchain.cache import InMemoryCache
#from langchain.globals import set_llm_cache
#set_llm_cache(InMemoryCache())


In [2]:

def resolve_direct(sitation, channel=ChatChannel, max_steps_per_player=7):
    players = sitation.players
    link(players)
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message_to_all(sender, others, [sitation.judge])
        if sitation.agreement_check():
            return


def resolve_with_mediator(sitation, mediator_key, channel=ChatChannel, max_steps_per_player=7):
    names = and_join(p.name for p in sitation.players)
    mediator = sitation.factory.create_persona(
        mediator_key,
        public=f"$name is a mediator to arrange arrange an agreement between {names}",
        goal=f"to help {names} find an agreement on {sitation.subject}; you will not watch the movie")
    players = sitation.players
    link(players + [mediator])
    for i in range(max_steps_per_player):
        for j, sender in enumerate(players):
            others = players[:j] + players[j+1:]
            channel.query_and_send_message(sender, mediator, others + [sitation.judge])
        channel.query_and_send_message_to_all(mediator, players, [sitation.judge])
        if sitation.agreement_check():
            return


# with TracingNode("root") as root:
#     resolver = Sitation("a specific movie name they want to watch together", factory, ["Maria-gpt35", "Connor-gpt35"])

#     with TracingNode("resolving"):
#         resolve_direct(resolver, "Edward-gpt35")
#     print(resolver.agreement_result)
#     with TracingNode("scoring"):
#         scoring = resolver.score_result()
#     df = pd.DataFrame(scoring, columns=["key", "result", "score_basic", "score_wh"])
#     print(df)
    
#root.display()

In [3]:
storage = FileStorage("logs")
storage.start_server()

<ServerHandle http://localhost:37573>

In [4]:
@dataclass
class Experiment:
    name: str
    people: str
    subject: str
    repeats: int
    permutations: bool
    groups: list[list[str]]

def experiments_from_yaml(path):
    with open(path) as f:
        data = f.read()        
    experiments = from_yaml(list[Experiment], data)
    return {e.name: e for e in experiments}

experiments = experiments_from_yaml("experiments.yaml")
experiments

{'movies': Experiment(name='movies', people='people-movies.yaml', subject='a single specific movie name they want to watch together', repeats=2, permutations=True, groups=[['Maria-gpt35', 'Jane-gpt35'], ['Don-gpt35', 'Jane-gpt35'], ['Don-gpt35', 'Jane-gpt35'], ['Edward-gpt35', 'Connor-gpt35'], ['Edward-gpt35', 'Jane-gpt35'], ['Lilly-gpt35', 'Edward-gpt35']]),
 'board_games': Experiment(name='board_games', people='people-bg.yaml', subject='a single specific board game to play together', repeats=2, permutations=True, groups=[['Carl-gpt35', 'Emma-gpt35']])}

In [10]:
def resolve_situation(situation, method):
    with TracingNode("resolving") as node:
        method(situation)
        node.set_result(situation.agreement_result)
    if situation.agreement_result is None:
        return None
    with TracingNode("scoring"):
        scoring = situation.score_result()
        return pd.DataFrame(scoring, columns=["person_key", "result", "score_basic", "score_wh"])    

def run_experiment(exp):
    result = []
    with TracingNode(f"Experiment {exp.name}") as root:
        factory = Factory.from_yaml(exp.people)
        for group in exp.groups:
            if exp.permutations:
                g_iter = itertools.permutations(group)
            else:
                g_iter = iter([group])
            for people in g_iter:
                for method_name, method_fn in [("mediator", lambda s: resolve_with_mediator(s, "Mediator")),
                                               ("direct", resolve_direct),
                                              ]:
                    for _ in range(exp.repeats):
                        uid = uuid.uuid4().hex
                        with TracingNode(f"experiment instance {uid}", inputs={
                            "uid": uid,
                            "method": method_name,
                            "experiment": exp.name,
                            "people": people
                        }):
                            situation = Situation(exp.subject, factory, people)    
                            df = resolve_situation(situation, method_fn)
                            if df is not None:
                                df["method"] = method_name
                                df["experiment"] = exp.name
                                df["instance"] = uid
                                df["group"] = ",".join(people)
                                result.append(df)
    return pd.concat(result)

with TracingNode("root", storage=storage) as root:
    df = run_experiment(experiments["board_games"])
    df.to_csv("out3.csv", index=False)
df

,person_key,result,score_basic,score_wh,method,experiment,instance,group
0,Carl-gpt35,Splendor,3.0,9.0,mediator,board_games,c716cb6e3d49483fb2e2da4941fcd8ee,"Carl-gpt35,Emma-gpt35"
1,Emma-gpt35,Splendor,9.0,9.0,mediator,board_games,c716cb6e3d49483fb2e2da4941fcd8ee,"Carl-gpt35,Emma-gpt35"
0,Carl-gpt35,7 Wonders,3.0,9.0,mediator,board_games,18c1fbe4281345d4b35692d26932391c,"Carl-gpt35,Emma-gpt35"
1,Emma-gpt35,7 Wonders,8.0,10.0,mediator,board_games,18c1fbe4281345d4b35692d26932391c,"Carl-gpt35,Emma-gpt35"
0,Carl-gpt35,Sushi Go!,8.0,10.0,direct,board_games,0a596009efe04041a7b17ccc0e630db7,"Carl-gpt35,Emma-gpt35"
1,Emma-gpt35,Sushi Go!,8.0,9.0,direct,board_games,0a596009efe04041a7b17ccc0e630db7,"Carl-gpt35,Emma-gpt35"
0,Carl-gpt35,Azul,6.0,8.0,direct,board_games,d00b3465da5a4ff0b310bd06d0010c45,"Carl-gpt35,Emma-gpt35"
1,Emma-gpt35,Azul,9.0,9.0,direct,board_games,d00b3465da5a4ff0b310bd06d0010c45,"Carl-gpt35,Emma-gpt35"
0,Emma-gpt35,7 Wonders,9.0,9.0,mediator,board_games,e06b84b71d244bdfa061e4fdb0bb78cb,"Emma-gpt35,Carl-gpt35"
1,Carl-gpt35,7 Wonders,5.0,9.0,mediator,board_games,e06b84b71d244bdfa061e4fdb0bb78cb,"Emma-gpt35,Carl-gpt35"
